web scrapping top n popular movie titles including casts, directors, actors, and image from imdb


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
"""change this to decide top n popular movie titles in imdb
please note this script may detect duplicate data and output may be less than expected
please note that output from the day you run the script and another day may different
because imdb1s algorithm keep changing their popular movie list"""
count:int=10

In [ ]:
driver = webdriver.Firefox()
driver.get("https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm")
elem = driver.find_element(By.CLASS_NAME, "lister-list")
x=elem.get_attribute('innerHTML')
driver.close()

imdb - listerlistclass html tag

In [ ]:
print(x)

finding all image covers for top 100 movies

In [ ]:
import regex as re

In [ ]:
img=re.findall("(?<=img src=\").*?(?=.jpg\")",x)
print(img)

finding link to go into better movie details

In [ ]:
ahref=re.findall("(?<=a href=\").*?(?=/?pf)",x)
ahref=[i[:-2] for i in ahref]
print(ahref)

finding directors, writers, and casts name

In [ ]:
list_of_names=[]
titles=[]


In [ ]:
def findPeople(d):
    names={}
    directors=[]
    writers=[]
    casts=[]

    for i in range(count):
        x=re.findall(f"(?<=dr{i}\"> ).*?(?= </a>)",d)
        if len(x)>0:
            directors.append(x)
    for i in range(count):
        x=re.findall(f"(?<=wr{i}\"> ).*?(?= </a>)",d)
        if len(x)>0:
            writers.append(x)
    for i in range(count):
        x=re.findall(f"(?<=cr{i}\"> ).*?(?= </a>)",d)
        if len(x)>0:
            casts.append(x)

    names["directors"]=directors
    names["writers"]=writers
    names["casts"]=casts
    list_of_names.append(names)


In [ ]:
def getTitle(d):
    titles.append(re.findall(f"(?<=url\">).*?(?=</a>)",d))

In [ ]:
c=[]
driver=webdriver.Firefox()
for i in range(count):
    driver.get("https://www.imdb.com/"+ahref[i]+"/fullcredits/")
    credits=driver.find_element(By.ID,"wrapper")
    c.append(credits.get_attribute('innerHTML'))

driver.close()
    
for i in range(count):
    d=re.sub('\s+',' ',c[i])
    findPeople(d)
    getTitle(d)

In [ ]:
movies={"titles":titles,"names":list_of_names,"images":img}

In [ ]:
movies

Overview

In [ ]:
a,b,c=pd.DataFrame(movies["titles"]),pd.DataFrame(movies["names"]),pd.DataFrame(movies["images"])

In [ ]:
a=a.drop_duplicates()
a

In [ ]:
b

In [ ]:
c

Data Cleaning

In [ ]:
temp=movies["names"]
directors=[]
writers=[]
casts=[]

def outFromNestedList(n):
    t=[]
    for i in n:
        t.append(",".join(i))
    s=",".join(t)
    return s

for i in range(count):
    director_name=outFromNestedList(temp[i]["directors"])
    writer_name=outFromNestedList(temp[i]["writers"])
    cast_name=outFromNestedList(temp[i]["casts"])
    directors.append(director_name)
    writers.append(writer_name)
    casts.append(cast_name)
    

names=pd.DataFrame([directors,casts,writers])
names=names.transpose()
names

In [ ]:
names=names.drop_duplicates()


In [ ]:
names=names.rename(columns={0:"directors",1:"casts",2:"writers"})
names

In [ ]:
title=a
image=img[0:len(title)]
image=pd.DataFrame(image)

In [ ]:
title=pd.DataFrame(movies["titles"])

In [ ]:
title=title.drop_duplicates()
name=names.drop_duplicates()
image=image.drop_duplicates()

In [ ]:
title=title.reset_index()
name=name.reset_index()
image=image.reset_index()

In [ ]:
title=title.drop(columns="index")
name=name.drop(columns="index")
image=image.drop(columns="index")

In [ ]:
title["director"]=name["directors"]
title["cast"]=name["casts"]
title["writer"]=name["writers"]
title=title.rename(columns={0:"Movie Name"})

In [ ]:
title